In [36]:
from fastai import *
from fastai.text import *
from fastai.text.learner import _model_meta
import pandas as pd
import os
from pathlib import Path

In [2]:
path = Path('/home/louis/Documents/NLP_satl/data')

In [3]:
path.ls()

[PosixPath('/home/louis/Documents/NLP_satl/data/train_full_m_shuffled.csv'),
 PosixPath('/home/louis/Documents/NLP_satl/data/models'),
 PosixPath('/home/louis/Documents/NLP_satl/data/vocab_lm.pkl'),
 PosixPath('/home/louis/Documents/NLP_satl/data/full.txt'),
 PosixPath('/home/louis/Documents/NLP_satl/data/exp'),
 PosixPath('/home/louis/Documents/NLP_satl/data/ll_clas_sample.pkl'),
 PosixPath('/home/louis/Documents/NLP_satl/data/train_full.tsv'),
 PosixPath('/home/louis/Documents/NLP_satl/data/.gitignore'),
 PosixPath('/home/louis/Documents/NLP_satl/data/data_cals_wt_2'),
 PosixPath('/home/louis/Documents/NLP_satl/data/data_class_preproc'),
 PosixPath('/home/louis/Documents/NLP_satl/data/test.csv'),
 PosixPath('/home/louis/Documents/NLP_satl/data/my_models'),
 PosixPath('/home/louis/Documents/NLP_satl/data/ll_lm.pkl'),
 PosixPath('/home/louis/Documents/NLP_satl/data/info.pkl'),
 PosixPath('/home/louis/Documents/NLP_satl/data/twitter-datasets.zip'),
 PosixPath('/home/louis/Documents/NLP_

In [58]:
with open(path/'twitter-datasets'/'train_neg_full.txt', 'r') as f :
    neg_tweets = f.readlines()

In [59]:
with open(path/'twitter-datasets'/'train_pos_full.txt', 'r') as f :
    pos_tweets = f.readlines()

In [60]:
len(neg_tweets), len(pos_tweets)

(1250000, 1250000)

In [61]:
df = pd.DataFrame({'tweet' : neg_tweets + pos_tweets, 'label' : ['neg' for i in range(len(neg_tweets))] + ['pos' for i in range(len(pos_tweets))]})

In [63]:
ts = df['tweet'].apply(lambda x : len(x.split(' ')))


In [64]:
ts.max()

128

In [62]:
df.head()

,tweet,label
0,vinco tresorpack 6 ( difficulty 10 of 10 objec...,neg
1,glad i dot have taks tomorrow ! ! #thankful #s...,neg
2,1-3 vs celtics in the regular season = were fu...,neg
3,<user> i could actually kill that girl i'm so ...,neg
4,<user> <user> <user> i find that very hard to ...,neg


In [31]:
df.shape

(2500000, 2)

In [32]:
df.to_csv(path/'tweets.csv')

In [34]:
#data_lm = TextLMDataBunch.from_csv(path, 'tweets.csv')

In [35]:
#data_lm.save('data_lm_export.pkl')

In [4]:
data_lm = load_data(path,'data_lm_export.pkl' )

FileNotFoundError: [Errno 2] No such file or directory: '/home/louis/Documents/NLP_satl/data/data_lm_export.pkl'

In [13]:
data_lm.show_batch()

idx,text
0,"lmaooo ! okay ! advertise that twittaaa anywhere / anytime \n xxbos a plump and perky turkey ( paperback plump and perky turkey is a marshall cavendish publication . < url > \n xxbos < user > yeah , you guys should it was fun \n xxbos < user > xxunk woof * grabs a handful of ur bottom * u'll do for me \n xxbos"
1,"> do n't you think sex was enough ? no need to introduce architecture in the same conversation . just go and have that drink . \n xxbos < user > today is the day \n xxbos i love waking up next to this boy < url > \n xxbos we all have targets , # usain bolts is to be a living legend "" a living"
2,"xxunk \n xxbos < user > nooo .. i want him so bad tho .. like he just so chill and sweet \n xxbos < user > i 'm waiting on a recipe for honey bun cake \n xxbos turner clc utah utes notepads , 5 x 8 inches , 2 packs ( 8 xxunk the official collegiate 2 - pack notepads is the p ... <"
3,"kill me soon "" < user > mosquitoes have killed more humans than all the wars in history . "" \n xxbos < user > who said i liked u ? \n xxbos girlfriend , boyfriend , friend everythinq has an end except for family # xxunk ! \n xxbos < user > thanks lmao ! i tried \n xxbos rey go to the hospital \n"
4,", she tryna jerk meh \n xxbos < user > tu xxunk out of twitter ! ... have a nice day ! \n xxbos "" batman is n't afraid to beat up a girl , he beats up cat girl ! "" "" ... i play cat woman on batman arkham city "" "" but i would n't beat you up "" \n xxbos like this shit"


In [14]:
#data_clas = TextClasDataBunch.from_csv(path, 'tweets.csv', vocab=data_lm.train_ds.vocab, bs=32, text_cols='tweet', label_cols='label')

In [15]:
data_clas.save('data_clas_export.pkl')

In [ ]:
DataBunch.save()

In [16]:
data_clas.show_batch()

text,target
xxbos party at your house ? bap bap ! lml rt < user > my birthday is in 10 days ! ! ! \ 355 \ 240 \ 274 \ 355 \ 276 \ 211 \ 355 \ 240 \ 274 \ 355 \ 276 \ 210 \ 355 \ 240 \ 274 \ 355 \ 276 \ 211 \ 355 \ 240 \ 274 \ 355 \ 276 \,pos
"xxbos # < - < ) i was gunna give you this waffle , ( > - ) > # but then i was like , ( > # ' < ) i m hungry ( > - ' < ) so i ate it \n",neg
xxbos how school works : 2 + 2 = 4 . homework : 2 + 4 + 2 = 8 . exam : john had 4 apples . he eats one and gives one to a friend . calculate the suns mass ... \n,neg
xxbos microstar int . xxunk - v2 2 gb memory ram kit ( 2x1 gb ) ( a - tech brand 2 gb ddr - 266 ( pc2100 ) kit ( 2x1 gb ) for micros ... < url > \n,neg
xxbos when u hope for a hot guy to sit next to u on a plane ... and u get creepy xxunk ! ! then hot guys smiles and gives u the ' i 'm sorry ' face ! \n,neg


In [3]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

NameError: name 'data_lm' is not defined

In [6]:
learn.layer_groups

[Sequential(
   (0): WeightDropout(
     (module): LSTM(400, 1150, batch_first=True)
   )
   (1): RNNDropout()
 ), Sequential(
   (0): WeightDropout(
     (module): LSTM(1150, 1150, batch_first=True)
   )
   (1): RNNDropout()
 ), Sequential(
   (0): WeightDropout(
     (module): LSTM(1150, 400, batch_first=True)
   )
   (1): RNNDropout()
 ), Sequential(
   (0): Embedding(60007, 400, padding_idx=1)
   (1): EmbeddingDropout(
     (emb): Embedding(60007, 400, padding_idx=1)
   )
   (2): LinearDecoder(
     (decoder): Linear(in_features=400, out_features=60007, bias=True)
     (output_dp): RNNDropout()
   )
 )]

In [7]:
learn.layer_groups[0]

Sequential(
  (0): WeightDropout(
    (module): LSTM(400, 1150, batch_first=True)
  )
  (1): RNNDropout()
)

In [19]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.117996,3.913267,0.357143,28:27


In [20]:
learn.predict("This is a review about", n_words=10)

'This is a review about rumors behind our winning everyones night ! round ... cap'

In [21]:
learn.save_encoder('ft_enc')

In [22]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')

In [23]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.522422,0.443533,0.783238,13:31


In [24]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))


epoch,train_loss,valid_loss,accuracy,time
0,0.377730,0.318243,0.859916,16:18


In [25]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))



epoch,train_loss,valid_loss,accuracy,time
0,0.318376,0.287228,0.874658,36:28


In [36]:
learn.predict('I am happy')

(Category pos, tensor(1), tensor([0.0527, 0.9473]))

In [27]:
learn.save('first_fastai')

In [33]:
learn.data.show_batch()

text,target
xxbos party at your house ? bap bap ! lml rt < user > my birthday is in 10 days ! ! ! \ 355 \ 240 \ 274 \ 355 \ 276 \ 211 \ 355 \ 240 \ 274 \ 355 \ 276 \ 210 \ 355 \ 240 \ 274 \ 355 \ 276 \ 211 \ 355 \ 240 \ 274 \ 355 \ 276 \,pos
xxbos b . o . b . so hard to breathe ( itunes b . o . b . so hard to breathe ( itunes ) download : b . o . b . so hard to breathe ... < url > \n,neg
"xxbos * sips acid * "" < user > < user > sorry oh ... please ... is that u in ur avatar ? i hear it is . if so , do n't u value or rate urself ? "" \n",neg
xxbos microstar int . xxunk combo - fr ( ms xxunk - 010 ) 1 gb memory ram upgrade ( a - tech brand 1 gb ddr 2 - 533 ( pc2 - 4200 ) for mi ... < url > \n,neg
xxbos 4.5 k tweet gives a s / o to my < user > ! i miss you so much it 's crazy . ca n't wait for you to come back ! hope you 're havin fun there tho ! \n,pos


In [37]:
with open(path/'test_data.txt', 'r') as f:
    test_data = f.readlines()

In [38]:
test_data[:10]

['1,sea doo pro sea scooter ( sports with the portable sea-doo seascootersave air , stay longer in the water and ... <url>\n',
 "2,<user> shucks well i work all week so now i can't come cheer you on ! oh and put those batteries in your calculator ! ! !\n",
 '3,i cant stay away from bug thats my baby\n',
 "4,<user> no ma'am ! ! ! lol im perfectly fine and not contagious anymore lmao\n",
 '5,whenever i fall asleep watching the tv , i always wake up with a headache\n',
 "6,<user> he needs to get rid of that thing ! it scares me lol but he don't need a car either . he needs drivers ed again .\n",
 '7,its whatever . in a terrible mood ( (\n',
 "8,yesss ! rt <user> <user> thanks jordan , i love you and i'm gonna call you later !\n",
 '9,my friend <user> text me to check up on me last night .\n',
 '10,<user> #followback please . when will ur #unitytour come to europe and sweden ? ?\n']

In [45]:
learn.predict(test_data[0].split(',', 1)[1])[1].item()

0

In [46]:
splitted_test_data = [(eval(s.split(',', 1)[0]), s.split(',', 1)[1], -1 if learn.predict(s.split(',', 1)[1])[1].item()==0 else 1) for s in test_data]

In [48]:
df_test_pred = pd.DataFrame({'Id' : [r[0] for r in splitted_test_data], 'Prediction' : [r[2] for r in splitted_test_data]})

In [49]:
df_test_pred.head()

,Id,Prediction
0,1,-1
1,2,-1
2,3,-1
3,4,1
4,5,-1


In [51]:
df_test_pred.to_csv('first_fastai_try.csv', index=False)

In [52]:
learn.save(path/'first_fastai.pkl')

# With preprocessed tweets

In [66]:
n_tweets = df.shape[0]
permutation = np.random.permutation(n_tweets)
pctg = 0.2
train_idxs = permutation[int(pctg*n_tweets):]
valid_idxs = permutation[:int(pctg*n_tweets)]
train_df = df.iloc[train_idxs]
valid_df = df.iloc[valid_idxs]

In [71]:
df = df.drop(index=1857314)

In [73]:
df.iloc[1857314]

label                                                    1
tweet    going to dollywood with my bestfriend in a few...
Name: 1857315, dtype: object

In [68]:
ts = df['tweet'].apply(lambda x : len(x.split(' ')))


In [69]:
ts.max()

3827

In [70]:
ts[ts == ts.max()]

1857314    3827
Name: tweet, dtype: int64

In [54]:
ts[ts > 10*ts.std()].shape

(1261,)

In [41]:
df.shape

(2458295, 2)

In [24]:
ts[ts == ts.max()]

1857314    3827
Name: tweet, dtype: int64

In [21]:
train_df.head()

,label,tweet
564117,0,i wish you could recover the last couple pictu...
15077,0,"cables , diamonds , & herringbone : secrets of..."
165703,0,<hashtag> # 51days maybe good for the belieber...
1455781,1,<user> i noe . <repeat> but it kinda . <repeat...
1756858,1,"<user> aww , thanks qt you 're so sweet . <hea..."


In [19]:
n_tweets, train_df.shape[0], valid_df.shape[0]

(2458295, 1966636, 491659)

In [16]:
#data_lm = TextLMDataBunch.from_df(path, train_df, valid_df)
#data_lm.save('data_lm_preproc_fastai.pkl')

In [36]:
data_lm = load_data(path,'data_lm_preproc_fastai.pkl' )

In [40]:
v = data_lm.vocab

In [41]:
v

In [18]:
data_lm.show_batch()

idx,text
0,"< number > likes . < repeat > girl with boobs falling out : "" lol i just sneezed "" - < number > likes . < sadface > xxbos < user > because he is a christian . xxbos < user > what about you anything exciting planned ? x xxbos < user > < number > yrs this makes me feel so old , remember wen it came"
1,"no filling , xxunk xxbos say hi to tish please ? ( < user > live on < url > xxbos everytime i see something about < hashtag > # xxunk it makes me tear up xxbos < user > that 's very kind of you . btw i hope mrs yg is making good progress . ) ) < hashtag > # xxunk xxbos amo < elong > xxunk"
2,meet her soon xxbos girls that always wanna go through your phone - . - obviously i ' m going to delete what should n't be there xxbos rt < user > i could go for a xxunk right now ! < hashtag > # xxunk but really xxbos < user > loool why am i missing this xxbos < user > hahaha ! hope your not talking about me
3,xxbos < user > haha wel < elong > maybe not eh but there is has to be like there 's so many people out there i ' m just one fea :} haha xxbos xxunk xxunk jiten = < smile > ictionary of today 's design ( japanese edition < url > xxbos good morning tweeps ! have a great wednesday ahead . be real and be safe god
4,"meant ! you ' ve forgotten me i see how it is xxbos < hashtag > # yougetmajorpointsif you have ambition , goals , and are going somewhere with your life . but still have fun . aye xxbos pretty close to my baby < url > xxbos best night with my bestie for lifee do n't know what i would do with out < user > < hashtag >"


In [5]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [9]:
learn.unfreeze()

In [6]:
learn.validate()

[8.0450945, tensor(0.0910)]

In [9]:
learn.fit_one_cycle(1, 0.01)

epoch,train_loss,valid_loss,accuracy,time
0,4.073377,3.867702,0.355777,31:08


In [10]:
learn.save('first_epoch')

In [11]:
learn.freeze_to(-2)

In [12]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.702616,3.581247,0.394768,31:45


In [13]:
learn.save('second_epoch')

In [14]:
learn.unfreeze()

In [15]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.517644,3.437596,0.412417,34:51


In [16]:
learn.save('full_lm_learner_3e')

In [27]:
learn.save_encoder('encoder1')

In [7]:
#data_clas = TextClasDataBunch.from_csv(path, 'train_full_m_shuffled.csv', vocab=data_lm.train_ds.vocab, text_cols='tweet', label_cols='label')
#data_clas.save('data_class_preproc_csv')

In [3]:
data_clas = load_data(path, 'data_class_preproc_csv')

In [4]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('encoder1')

In [8]:
learn = learn.load(path/'models'/'clas_three_epoch')

In [9]:
learn.validate()

[0.3050526, tensor(0.8823)]

In [14]:
learn.save('clas_frist_epoch')

In [16]:
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.306563,0.280011,0.877928,14:34


In [17]:
learn.save('clas_second_epoch')

In [10]:
learn.unfreeze()

In [11]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.291260,0.258058,0.889428,24:18


In [12]:
learn.save('clas_four_epoch')

In [13]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.277209,0.255365,0.891044,25:38


In [14]:
learn.save('clas_fve_epoch')

In [4]:
#data_cals_wt = TextClasDataBunch.from_csv(path, 'train_full_m_shuffled.csv', test='test.csv', vocab=data_lm.train_ds.vocab, text_cols='tweet', label_cols='label')
#data_cals_wt.save('data_cals_wt')

In [4]:
data_cals_wt = load_data(path, 'data_cals_wt')

In [5]:
learn = text_classifier_learner(data_cals_wt, AWD_LSTM, drop_mult=0.5)


In [9]:
learn = learn.load(path/'models'/'clas_fve_epoch')

In [12]:
learn.unfreeze()

In [9]:
preds = learn.get_preds(DatasetType.Test)

In [8]:
learn.validate()

[0.24701734, tensor(0.8955)]

In [19]:
learn.data.test_ds[4]

(Text xxbos whenever i fall asleep watching the tv , i always wake up with a headache 
  , EmptyLabel )

In [14]:
clas = preds[0].argmax(dim=1)

In [15]:
clas = [-1 if i == 0 else 1 for i in clas ]

In [22]:
learn.data.one_item(':')

(tensor([[   2,   53,    7, 2262,   22,    7, 2262,   76]], device='cuda:0'),
 tensor([0], device='cuda:0'))

(Category 1, tensor(1), tensor([0.0563, 0.9437]))

In [30]:
preds = [learn.predict(s)[1].item() for s in tweets]

In [37]:
submission.to_csv('submission_fa.csv', index=False)

In [33]:
preds = [-1 if i == 0 else 1 for i in preds]

In [36]:
submission = pd.DataFrame({'Id' : [i+1 for i in range(len(preds))], 'Prediction' : preds})

In [38]:
learn.unfreeze()

In [39]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.273110,0.249885,0.893546,24:58
1,0.262698,0.244373,0.896138,25:04


In [40]:
learn.validate()

[0.24437328, tensor(0.8961)]

In [16]:
from tqdm import tqdm_notebook

In [47]:
iterator = tqdm_notebook(tweets, total=len(tweets), desc='Test predictions')
preds_2 = []
for t in iterator :
    if learn.predict(t)[1].item() == 0 :
        preds_2.append(-1)
    else :
        preds_2.append(1)

In [51]:
learn.save('lots_epoch')

In [49]:
preds_2[:10]

[-1, -1, 1, 1, -1, -1, -1, 1, 1, 1]

In [23]:
submission = pd.DataFrame({'Id' : [i+1 for i in range(len(preds_2))], 'Prediction' : preds_2})

NameError: name 'preds_2' is not defined

In [53]:
submission.to_csv('submission_fa_2.csv', index=False)

# ----

In [9]:
#data_cals_wt = TextClasDataBunch.from_csv(path, 'train_full_m_shuffled.csv', valid_pct=0.001, test='test.csv', vocab=data_lm.train_ds.vocab, text_cols='tweet', label_cols='label')
#data_cals_wt.save('data_cals_wt_2')

In [3]:
data_clas_wt_2 = load_data(path, 'data_cals_wt_2')

In [5]:
learn = text_classifier_learner(data_clas_wt_2, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('encoder1')

In [6]:
learn.freeze_to(-1)

In [7]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.390637,0.341169,0.844462,10:27


In [8]:
learn.save('2_first_epoch')

In [9]:
learn.freeze_to(-2)

In [10]:
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))


epoch,train_loss,valid_loss,accuracy,time
0,0.305778,0.264726,0.882047,12:29


In [11]:
learn.save('2_second_epoch')

In [12]:
learn.unfreeze()

In [13]:
learn.fit_one_cycle(5, slice(2e-3/100, 2e-3))


epoch,train_loss,valid_loss,accuracy,time
0,0.298222,0.257615,0.883647,31:03
1,0.283480,0.249632,0.890044,27:45
2,0.276189,0.242331,0.899640,29:51
3,0.263866,0.236234,0.893643,28:50
4,0.246911,0.234757,0.900840,30:58


In [14]:
learn.save('2_5_aft_unfreeze')

In [18]:
df_test  = pd.read_csv('data/test.csv')

In [8]:
learn = load_learner(path/'models', '2_5_aft_unfreeze')

FileNotFoundError: [Errno 2] No such file or directory: '/home/louis/Documents/NLP_satl/data/models/2_5_aft_unfreeze'

In [ ]:
learn

In [19]:
tweets = df_test['tweet'].values

In [20]:
iterator = tqdm_notebook(tweets, total=len(tweets), desc='Test predictions')
preds_3 = []
for t in iterator :
    if learn.predict(t)[1].item() == 0 :
        preds_3.append(-1)
    else :
        preds_3.append(1)

In [22]:
preds_3[:10]

[-1, -1, 1, 1, -1, -1, -1, 1, 1, 1]

In [24]:
submission = pd.DataFrame({'Id' : [i+1 for i in range(len(preds_3))], 'Prediction' : preds_3})

In [26]:
submission.to_csv('submission_fa_3.csv')

In [23]:
learn.unfreeze()

In [13]:
learn.opt.opt.param_groups[-1]['params']

[]

In [31]:
[t.shape for t in learn.opt.opt.param_groups[6]['params']]

[torch.Size([1600, 400]),
 torch.Size([1600, 1150]),
 torch.Size([1600]),
 torch.Size([1600])]

In [4]:
#data_cals_wt = TextClasDataBunch.from_csv(path, 'train_full_m_shuffled.csv', valid_pct=0.001, test='test.csv', vocab=Vocab(pickle.load(open('data/vocab_lm.pkl', 'rb'))), text_cols='tweet', label_cols='label')
#data_cals_wt.save('data_cals_wt_2_james')

In [6]:
data_cals_wt = load_data(path, 'data_cals_wt_2_james')

In [64]:
??load_data

In [16]:
vocab = pickle.load(open('data/vocab_lm.pkl', 'rb'))

In [17]:
len(vocab)

60005

In [27]:
from preprocessing import *
from training import *
from fastai.layers import CrossEntropyFlat
import pickle
from fastai.text import get_text_classifier
import dill

In [26]:
def my_text_classifier_learner(data:DataBunch, arch:Callable, vocab_l, c=2,bptt:int=70, max_len:int=70*20, config:dict=None, 
                            pretrained:bool=True, drop_mult:float=1., lin_ftrs:Collection[int]=None, 
                            ps:Collection[float]=None, **learn_kwargs) -> 'TextClassifierLearner':
    "Create a `Learner` with a text classifier from `data` and `arch`."
    model = get_text_classifier(arch, vocab_l, c, bptt=bptt, max_len=max_len,
                                config=config, drop_mult=drop_mult, lin_ftrs=lin_ftrs, ps=ps)
    meta = _model_meta[arch]
    learn = RNNLearner(data, model, split_func=meta['split_clas'], **learn_kwargs)
    if pretrained:
        if 'url' not in meta: 
            warn("There are no pretrained weights for that architecture yet!")
            return learn
        model_path = untar_data(meta['url'], data=False)
        fnames = [list(model_path.glob(f'*.{ext}'))[0] for ext in ['pth', 'pkl']]
        learn.load_pretrained(*fnames, strict=False)
        learn.freeze()
    return learn

In [28]:
ll = pickle.load(open('data/ll_clas.pkl', 'rb'))
vocab = pickle.load(open('data/vocab_lm.pkl', 'rb'))

In [29]:
traind_dl, valid_dl = get_clas_dls(ll.train, ll.valid, 64)

In [31]:
da = DataBunch(traind_dl, valid_dl)

In [50]:
da.path = 'data/'

In [56]:
da.device =  torch.device('cuda')

da.loss_func = CrossEntropyFlat()

da.train_ds.vocab = Vocab(vocab)

In [62]:
learn = my_text_classifier_learner(da, AWD_LSTM, len(vocab), drop_mult=0.5)
learn.load_encoder('lm_1enco')
learn.model = learn.model.cuda()

In [58]:
learn.freeze_to(-1)

In [63]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,time


RuntimeError: Expected object of backend CUDA but got backend CPU for argument #3 'index'

In [8]:
learn.save('2_first_epoch')

In [9]:
learn.freeze_to(-2)

In [10]:
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))


epoch,train_loss,valid_loss,accuracy,time
0,0.305778,0.264726,0.882047,12:29


In [11]:
learn.save('2_second_epoch')

In [12]:
learn.unfreeze()

In [13]:
learn.fit_one_cycle(5, slice(2e-3/100, 2e-3))


epoch,train_loss,valid_loss,accuracy,time
0,0.298222,0.257615,0.883647,31:03
1,0.283480,0.249632,0.890044,27:45
2,0.276189,0.242331,0.899640,29:51
3,0.263866,0.236234,0.893643,28:50
4,0.246911,0.234757,0.900840,30:58


In [14]:
learn.save('2_5_aft_unfreeze')

In [18]:
df_test  = pd.read_csv('data/test.csv')

In [8]:
learn = load_learner(path/'models', '2_5_aft_unfreeze')

FileNotFoundError: [Errno 2] No such file or directory: '/home/louis/Documents/NLP_satl/data/models/2_5_aft_unfreeze'